### **Construcción y entrenamiento de un modelo de lenguaje simple con una red neuronal**

Este proyecto sirve como una introducción al campo del modelado de lenguaje, enfocándose en la creación de un generador de texto diseñado para componer canciones de rap de los años 90.

Utilizaremos modelos n-gramas basados en histogramas, implementados a través de la herramienta *Natural Language Toolkit* (NLTK). Este enfoque nos permite construir histogramas reveladores, que iluminan las cadencias matizadas de las frecuencias y distribuciones de palabras.

Estos pasos iniciales sientan las bases para comprender las complejidades de los patrones lingüísticos. A medida que avancemos, entraremos en el dominio de las redes neuronales dentro del entorno de PyTorch. 

En este ámbito, diseñaremos una red neuronal *feedforward*, explorando conceptos como las capas de *embeddings*. También perfeccionaremos la capa de salida, adaptándola para un rendimiento óptimo en tareas de modelado de lenguaje.

A lo largo de este recorrido, exploraremos diversas estrategias de entrenamiento ycon tareas fundamentales del procesamiento de lenguaje natural (NLP), incluyendo la tokenización y el análisis de secuencias.


#### Configuración

Para este cuaderno, utilizaremos las siguientes librerías:

*   [`pandas`](https://pandas.pydata.org/) para la gestión de datos.  
*   [`numpy`](https://numpy.org/) para realizar operaciones matemáticas.  
*   [`sklearn`](https://scikit-learn.org/stable/) para funciones relacionadas con aprendizaje automático y flujos de trabajo de *aprendizaje automático*.  
*   [`seaborn`](https://seaborn.pydata.org/) para la visualización de datos.  
*   [`matplotlib`](https://matplotlib.org/) como herramienta adicional para la creación de gráficos.


#### Instalación de librerías requeridas

Todas las librerías necesarias ya están preinstaladas en el entorno de docker del curso. Sin embargo, si ejecutas los comandos de este cuaderno en un entorno de Jupyter diferente (por ejemplo, **Watson Studio** o **Anaconda**), necesitarás instalar estas librerías utilizando la celda de código que aparece a continuación.

<h4 style="color:red;">Después de instalar las librerías a continuación, por favor REINICIA EL KERNEL y ejecuta todas las celdas.</h4>

In [ ]:
#%%capture

#!mamba install -y nltk
#!pip install torchtext -qqq

#### Importación de las librerías requeridas

_Se recomienda importar todas las bibliotecas necesarias en un solo lugar (aquí):_


In [ ]:
import warnings
from tqdm import tqdm  # Barra de progreso para bucles

warnings.simplefilter('ignore')  # Ignora todas las advertencias
import time
from collections import OrderedDict  # Diccionario que mantiene el orden de inserción

import re  # Expresiones regulares

import numpy as np  # Operaciones numéricas eficientes
import matplotlib.pyplot as plt  # Visualización de datos
import pandas as pd  # Manipulación de datos en forma de tablas

# Descarga de recursos de tokenización para NLTK
import nltk
nltk.download('punkt')

# Importación de PyTorch y módulos relevantes
import torch
import torch.nn as nn  # Módulo de redes neuronales
import torch.nn.functional as F  # Funciones de activación, pérdidas, etc.
import torch.optim as optim  # Optimizadores
import string
import time

# Importación adicional para visualización de reducción de dimensión
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE  # t-SNE para visualización de vectores de alta dimensión

# También puedes usar esta sección para suprimir advertencias generadas por tu código:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn


#### Definición de funciones auxiliares

Elimina todos los caracteres que no sean parte de palabras (todo excepto números y letras)


In [ ]:
def preprocess_string(s):
    # Eliminar todos los caracteres que no sean parte de palabras (todo excepto números y letras)
    s = re.sub(r"[^\w\s]", '', s)
    # Reemplazar todas las secuencias de espacios en blanco sin dejar espacio
    s = re.sub(r"\s+", '', s)
    # Reemplazar dígitos sin dejar espacio
    s = re.sub(r"\d", '', s)

    return s

### **Modelado de lenguaje**

El modelado de lenguaje es un concepto fundamental dentro del campo del procesamiento de lenguaje natural (NLP) y la inteligencia artificial. Consiste en predecir la probabilidad de una secuencia de palabras dentro de un idioma dado. Este método tiene una naturaleza estadística y busca capturar los patrones, estructuras y relaciones que existen entre las palabras en un corpus de texto determinado.

En esencia, un modelo de lenguaje busca comprender las probabilidades asociadas con secuencias de palabras. Esta comprensión puede aprovecharse en una multitud de tareas de NLP, incluyendo, pero no limitándose a, la generación de texto, traducción automática, reconocimiento de voz, análisis de sentimientos, entre otras.

Consideremos las siguientes letras de una canción para ver si podemos generar una salida similar a partir de una palabra dada.

In [ ]:
song= """We are no strangers to love
You know the rules and so do I
A full commitments what Im thinking of
You wouldnt get this from any other guy
I just wanna tell you how Im feeling
Gotta make you understand
Never gonna give you up
Never gonna let you down
Never gonna run around and desert you
Never gonna make you cry
Never gonna say goodbye
Never gonna tell a lie and hurt you
Weve known each other for so long
Your hearts been aching but youre too shy to say it
Inside we both know whats been going on
We know the game and were gonna play it
And if you ask me how Im feeling
Dont tell me youre too blind to see
Never gonna give you up
Never gonna let you down
Never gonna run around and desert you
Never gonna make you cry
Never gonna say goodbye
Never gonna tell a lie and hurt you
Never gonna give you up
Never gonna let you down
Never gonna run around and desert you
Never gonna make you cry
Never gonna say goodbye
Never gonna tell a lie and hurt you
Weve known each other for so long
Your hearts been aching but youre too shy to say it
Inside we both know whats been going on
We know the game and were gonna play it
I just wanna tell you how Im feeling
Gotta make you understand
Never gonna give you up
Never gonna let you down
Never gonna run around and desert you
Never gonna make you cry
Never gonna say goodbye
Never gonna tell a lie and hurt you
Never gonna give you up
Never gonna let you down
Never gonna run around and desert you
Never gonna make you cry
Never gonna say goodbye
Never gonna tell a lie and hurt you
Never gonna give you up
Never gonna let you down
Never gonna run around and desert you
Never gonna make you cry
Never gonna say goodbye
Never gonna tell a lie and hurt you"""

#### Natural Language Toolkit (NLTK)


NLTK es, en efecto, una librería de código abierto ampliamente utilizada en Python, diseñada específicamente para diversas tareas de procesamiento de lenguaje natural (NLP). Proporciona un conjunto completo de herramientas, recursos y algoritmos que facilitan el análisis y la manipulación de datos del lenguaje humano. 

#### Tokenización

La tokenización, un concepto fundamental dentro del campo del procesamiento de lenguaje natural (NLP), implica el proceso detallado de dividir un cuerpo de texto en unidades discretas conocidas como *tokens*. Estos tokens pueden abarcar palabras, frases, oraciones o incluso caracteres individuales, dependiendo del nivel de granularidad deseado para el análisis. 

Para los fines de este proyecto, nos enfocaremos en la *tokenización de palabras*, una técnica ampliamente utilizada. Esta técnica trata cada palabra del texto como una entidad independiente. Las palabras, típicamente separadas por espacios o signos de puntuación, actúan como tokens en este enfoque. Es importante señalar que la tokenización de palabras presenta características versátiles, incluyendo el manejo de mayúsculas, símbolos y signos de puntuación.

Para lograr este objetivo, utilizaremos la función ```word_tokenize```. Durante este proceso, eliminaremos los signos de puntuación, los símbolos y las letras mayúsculas.



In [ ]:
from nltk.tokenize import word_tokenize
def preprocess(words):
    tokens = word_tokenize(words)
    tokens = [preprocess_string(w) for w in tokens]
    return [w.lower() for w in tokens if len(w) != 0 or not (w in string.punctuation)]

tokens = preprocess(song)

El resultado es una colección de tokens, en la que cada elemento de la variable ```tokens``` corresponde a las letras de la canción, ordenados secuencialmente.


In [ ]:
tokens[0:10]

La distribución de frecuencias de palabras en una oración representa cuántas veces aparece cada palabra en esa oración en particular. Proporciona un conteo de las apariciones de palabras individuales, lo que permite entender cuáles son las palabras más comunes o frecuentes dentro de la oración dada. 

Trabajemos con el siguiente ejemplo sencillo:

```Texto```: **I like dogs and I kinda like cats**

```Tokens```: **[I like, dogs, and, I, kinda, like, cats]**

La función ```Count``` contabilizará las apariciones de las palabras en el texto de entrada.

$Count(\text{"I"})=2$

$Count(\text{"like"})= 2$

$Count(\text{"dogs"})=1$

$Count(\text{"and"})=1$

$Count(\text{"kinda"})=1$

$Count(\text{"cats"})=1$

$\text{Total de palabras} =8$


Utiliza ```FreqDist``` de NLTK para transformar la distribución de frecuencias de palabras. El resultado es un diccionario de Python donde las claves corresponden a las palabras y los valores indican la frecuencia con la que aparece cada palabra. 

Consideremos el siguiente ejemplo:


In [ ]:
# Crea una distribución de frecuencias de las palabras
fdist = nltk.FreqDist(tokens)
fdist

Dibujamos las diez palabras con mayor frecuencia.


In [ ]:
plt.bar(list(fdist.keys())[0:10], list(fdist.values())[0:10])
plt.xlabel("Palabras")
plt.ylabel("Frecuencia")
plt.show()

#### **Modelo unigrama**

Un modelo *unigrama* (Unigram Model) es un tipo simple de modelo de lenguaje que considera cada palabra en una secuencia de forma independiente, sin tener en cuenta las palabras anteriores. En otras palabras, modela la probabilidad de que cada palabra ocurra en el texto, sin importar que palabra la precede. 

Los modelos unigrama pueden verse como un caso especial de los modelos *n-grama*, donde *n* es igual a 1.

Podemos pensar que el texto sigue patrones y que las probabilidades se usan para medir qué tan probable es una secuencia de palabras. En un modelo unigrama, cada palabra se considera de forma independiente y no depende de las demás. 

Calculemos la probabilidad de **'I like tiramisu but I love cheesecake more'**.

$  P(\text{"I"}) = \frac{\text{Count}(\text{"I"})}{\text{Total de palabras}}=\frac{2}{8} = 0.250  $

$  P(\text{"like"}) = \frac{\text{Count}(\text{"like"})}{\text{Total de palabras}}=\frac{1}{8} = 0.125  $

$  P(\text{"tiramisu"}) = \frac{\text{Count}(\text{"tiramisu"})}{\text{Total de palabras}}=\frac{1}{8} = 0.125  $

$  P(\text{"but"}) = \frac{\text{Count}(\text{"but"})}{\text{Total de palabras}}=\frac{1}{8} = 0.125  $

$  P(\text{"I"}) = \frac{\text{Count}(\text{"I"})}{\text{Total de palabras}}=\frac{2}{8} = 0.250  $

$  P(\text{"love"}) = \frac{\text{Count}(\text{"love"})}{\text{Total de palabras}}=\frac{1}{8} = 0.125  $

$  P(\text{"cheesecake"}) = \frac{\text{Count}(\text{"cheesecake"})}{\text{Total de palabras}}=\frac{1}{8} = 0.125  $

$  P(\text{"more"}) = \frac{\text{Count}(\text{"more"})}{\text{Total de palabras}}=\frac{1}{8} = 0.125  $

$P(\text{"I"}, \text{"like"}, \text{"tiramisu"}, \text{"but"}, \text{"I"}, \text{"love"}, \text{"cheesecake"}, \text{"more"}) = P(\text{"I"}) \cdot P(\text{"like"}) \cdot P(\text{"tiramisu"}) \cdot P(\text{"but"}) \cdot P(\text{"I"}) \cdot P(\text{"love"}) \cdot P(\text{"cheesecake"}) \cdot P(\text{"more"}) = 0.250 \times 0.125 \times 0.125 \times 0.125 \times 0.250 \times 0.125 \times 0.125 \times 0.125$

En general, los modelos de lenguaje se reducen a predecir una secuencia de longitud $t$: $P(W_t, W_{t-1}, ..., W_0)$. En esta secuencia de ocho palabras, se tiene:

$P(W_7=\text{"more"}, W_6=\text{"cheesecake"}, W_5=\text{"love"}, W_4=\text{"I"}, W_3=\text{"but"}, W_2=\text{"tiramisu"}, W_1=\text{"like"}, W_0=\text{"I"})$

El subíndice sirve como un indicador posicional en la secuencia y no afecta la naturaleza de $P(\bullet)$. Al expresar formalmente la secuencia, la última palabra se posiciona a la izquierda, descendiendo gradualmente conforme se avanza en la secuencia.


Usando NLTK podemos normalizar los valores de frecuencia dividiéndolos por el conteo total de cada palabra para obtener una función de probabilidad. Ahora vamos a encontra la probabilidad de cada palabra.


In [ ]:
# Conteo total de cada palabra
C = sum(fdist.values())
C

Hallamos la probabilidad de la palabra _strangers_, es decir, $P(strangers)$.


In [ ]:
fdist['strangers'] / C

Además, obtenemos cada palabra individual convirtiendo los tokens en un conjunto (set).


In [ ]:
vocabulary = set(tokens)

#### Cómo el modelo unigrama predice la siguiente palabra probable

Consideremos un escenario del ejemplo anterior **"I like tiramisu but I love cheesecake more"**, donde se le pide al modelo unigrama predecir la siguiente palabra después de la secuencia **"I like"**.

Si la palabra con mayor probabilidad entre todas es **"I"**, con una probabilidad de 0.25, entonces, según el modelo, la palabra más probable después de **"I like"** sería **"I"**. Sin embargo, esta predicción no tiene sentido. Esto resalta una limitación importante del modelo unigrama: carece de contexto y sus predicciones dependen únicamente de la palabra con la probabilidad más alta, que en este caso es "I".

Incluso si varias palabras tienen la misma probabilidad más alta, el modelo elegirá aleatoriamente una de todas las opciones.


#### **Modelo bigrama**

Los bigramas representan pares de palabras consecutivas en una frase dada, es decir, $(w_{t-1}, w_t)$. Consideremos las siguientes palabras de tu ejemplo: "I like dogs and I kinda like cats."

La secuencia correcta de bigramas es:

$(I, like)$

$(like, dogs)$

$(dogs, and)$

$(and, I)$

$(I, kinda)$

$(kinda, like)$

$(like, cats)$


**Modelos 2-grama**: Los modelos bigrama utilizan la probabilidad condicional. La probabilidad de una palabra depende únicamente de la palabra anterior, es decir, se usa la probabilidad condicional $(W_{t}, W_{t-1})$ para predecir la probabilidad de que la palabra $W_t$ siga a la palabra $W_{t-1}$ en una secuencia. 

Podemos calcular la probabilidad condicional para un modelo bigrama siguiendo los siguientes pasos.


Realizamos el conteo bigrama para cada bigrama: $Count(W_{t-1}, W_{t})$

$Count(\text{I, like}) = 1$

$Count(\text{like, dogs}) = 1$

$Count(\text{dogs, and}) = 1$

$Count(\text{and, I}) = 1$

$Count(\text{I, kinda}) = 1$

$Count(\text{kinda, like}) = 1$

$Count(\text{like, cats}) = 1$


Ahora, calculemos la probabilidad condicional para cada bigrama en la forma de $P(w_{t} | w_{t-1})$, donde $w_{t-1}$ es el **contexto**, y el tamaño del contexto es 1.

$P(\text{"like"} | \text{"I"}) = \frac{\text{Count}(\text{"I, like"})}{\text{Total de ocurrencias de "I"}} = \frac{1}{2} = 0.5$

$P(\text{"dogs"} | \text{"like"}) = \frac{\text{Count}(\text{"like, dogs"})}{\text{Total de ocurrencias de "like"}} = \frac{1}{2} = 0.5$

$:$

$P(\text{"like"} | \text{"kinda"}) = \frac{\text{Count}(\text{"kinda, like"})}{\text{Total de ocurrencias de "kinda"}} = \frac{1}{1} = 1$

$P(\text{"cats"} | \text{"like"}) = \frac{\text{Count}(\text{"like, cats"})}{\text{Total de ocurrencias de "like"}} = \frac{1}{2} = 0.5$

Estas probabilidades representan la probabilidad de encontrar la segunda palabra en un bigrama, dada la presencia de la primera.


Este enfoque es, de hecho, una aproximación utilizada para determinar la palabra más probable $W_t$, dadas las palabras $W_{t-1}, W_{t-2}, \ldots, W_1$ en la secuencia.

$P(W_t | W_{t-1}, W_{t-2}, \ldots, W_1) \approx P(W_t | W_{t-1})$

La probabilidad condicional $P(W_t | W_{t-1})$ denota la probabilidad de encontrar la palabra $W_t$, basándose en el contexto proporcionado por la palabra precedente $W_{t-1}$. Al utilizar esta aproximación, se simplifica el proceso de modelado asumiendo que la ocurrencia de la palabra actual está principalmente influenciada por la palabra inmediatamente anterior en la secuencia. 

De forma general, se puede identificar la palabra más probable como:

$\hat{W_t} = \arg\max_{W_t} \left( P(W_t | W_{t-1}) \right)$


La función ```bigrams``` es una función proporcionada por la biblioteca NLTK en Python. Esta función toma una secuencia de tokens como entrada y devuelve un iterador sobre pares consecutivos de tokens, formando bigramas.


In [ ]:
bigrams = nltk.bigrams(tokens)
bigrams

Se convierte el generador en una lista, donde cada elemento de la lista es un bigrama.


In [ ]:
mi_bigrams = list(nltk.bigrams(tokens))

Podemos ver los primeros 10 bigramas.


In [ ]:
mi_bigrams[0:10]

Calculamos la distribución de frecuencias del bigrama $C(w_{t},w_{t-1})$ utilizando la función ```bigrams``` de NLTK.


In [ ]:
freq_bigrams = nltk.FreqDist(nltk.bigrams(tokens))
freq_bigrams

El resultado es similar a un diccionario, donde la clave es una tupla que contiene el bigrama.


In [ ]:
freq_bigrams[('we', 'are')]

Es posible mostrar los primeros 10 valores de la distribución de frecuencias.


In [ ]:
for mi_bigram in mi_bigrams[0:10]:
    print(mi_bigram)
    print(freq_bigrams[mi_bigram])

Aquí, podemos generar la distribución condicional normalizando la distribución de frecuencias de los unigrama. En este caso, lo haremos para la palabra 'strangers' y luego ordenamos los resultados:


In [ ]:
word = "strangers"
vocab_probabilities = {}
for next_word in vocabulary:
    vocab_probabilities[next_word] = freq_bigrams[(word, next_word)] / fdist[word]

vocab_probabilities = sorted(vocab_probabilities.items(), key=lambda x: x[1], reverse=True)

Se imprime las palabras que son más probables de ocurrir.


In [ ]:
vocab_probabilities[0:4]

Se crea una función para calcular la probabilidad condicional de $W_t$ dado $W_{t-1}$, ordena los resultados y devuélvelos como una lista.


In [ ]:
def make_predictions(mi_words, freq_grams, normlize=1, vocabulary=vocabulary):
    """
    Genera predicciones para la probabilidad condicional de la siguiente palabra dada una secuencia.

    Args:
        mi_words (list): Una lista de palabras en la secuencia de entrada.
        freq_grams (dict): Un diccionario que contiene las frecuencias de los n-gramas.
        normlize (int): Un factor de normalización para calcular las probabilidades.
        vocabulary (list): Una lista de palabras en el vocabulario.
    
    Returns:
        list: Una lista de las palabras predichas junto con sus probabilidades, ordenadas de forma descendente.
    """

    vocab_probabilities = {}  # Inicializa un diccionario para almacenar las probabilidades de las palabras predichas

    context_size = len(list(freq_grams.keys())[0])  # Determina el tamaño del contexto a partir de las claves de los n-gramas

    # Preprocesa las palabras de entrada y tomar sólo las palabras de contexto relevantes
    mi_tokens = preprocess(mi_words)[0:context_size - 1]

    # Calcula las probabilidades para cada palabra del vocabulario dado el contexto
    for next_word in vocabulary:
        temp = mi_tokens.copy()
        temp.append(next_word)  # Añade la siguiente palabra al contexto

        # Calcula la probabilidad condicional utilizando la información de frecuencia
        if normlize != 0:
            vocab_probabilities[next_word] = freq_grams[tuple(temp)] / normlize
        else:
            vocab_probabilities[next_word] = freq_grams[tuple(temp)]
    # Ordena las palabras predichas basándose en sus probabilidades de forma descendente
    vocab_probabilities = sorted(vocab_probabilities.items(), key=lambda x: x[1], reverse=True)

    return vocab_probabilities  # Devuelve la lista ordenada de palabras predichas y sus probabilidades

Establece $W_{t-1}$ a 'are' y luego calcula todos los valores de $P(W_t | W_{t-1}=are)$.


In [ ]:
mi_words = "are"

vocab_probabilities = make_predictions(mi_words, freq_bigrams, normlize=fdist['i'])

In [ ]:
vocab_probabilities[0:10]

La palabra con la mayor probabilidad, denotada como $\hat{W}_t$, es la que aparece en el primer elemento de la lista; esto se puede usar como una función de autocompletar simple:


In [ ]:
vocab_probabilities[0][0]

Generamos una secuencia utilizando el modelo bigrama, aprovechando la palabra previa _(t-1)_ para predecir y generar la siguiente palabra en la secuencia.


In [ ]:
mi_song = ""
for w in tokens[0:100]:
    mi_word = make_predictions(w, freq_bigrams)[0][0]
    mi_song += " " + mi_word

In [ ]:
mi_song

Creamos una secuencia con un modelo de n‑gramas, comenzando por la primera palabra y generando una salida inicial. A continuación, empleamos esa salida para predecir la siguiente palabra de la secuencia: introducimos una palabra en el modelo, usamos su salida para predecir la siguiente y repetimos el proceso.

In [ ]:
mi_song = "i"

for i in range(100):
    mi_word = make_predictions(mi_word, freq_bigrams)[0][0]
    mi_song += " " + mi_word

In [ ]:
mi_song

Este método puede no ofrecer resultados óptimos. Consideremos lo siguiente:

$\hat{W_1}=\arg\max_{W_1} \left( P(W_1 | W_{0}=\text{like})\right)$.

Al evaluarlo, observamos que el resultado para $\hat{W}_1$ incluye tanto "dogs" como "cats" con igual probabilidad.


####  **Modelo trigrama**

Para la oración del ejemplo: 'I like dogs and I kinda like cats'

$ (I, like, dogs) $

$(like, dogs, and) $

$(dogs, and, I)$

$(and, I, kinda)$

$(I, kinda, like)$

$(kinda, like, cats)$

Los modelos trigrama también incorporan la probabilidad condicional. La probabilidad de una palabra depende de las dos palabras precedentes. Se usa la probabilidad condicional $P(W_t | W_{t-2}, W_{t-1})$ para predecir la probabilidad de que la palabra $W_t$ siga a las dos palabras previas en una secuencia. El contexto es $W_{t-2}, W_{t-1}$ y su tamaño es 2. Calculemos la probabilidad condicional para cada trigrama:

Calculamos las frecuencias de cada trigrama: $Count(W_{t-2}, W_{t-1}, W_t)$

### Conteo de frecuencias de trigramas

$ \text{Count(I, like, dogs)} = 1 $

$ \text{Count(like, dogs, and)} = 1 $

$\text{Count(dogs, and, I)} = 1$

$ \text{Count(and, I, kinda)} = 1$

$ \text{Count(I, kinda, like)} = 1 $

$ \text{Count(kinda, like, cats)} = 1 $

La probabilidad condicional $ P(w_{t} | w_{t-1}, w_{t-2})$, donde $w_{t-1}$ y $w_{t-2}$ forman el contexto (de tamaño 2).

Para entender mejor cómo esto supera al modelo bigrama, calculemos las probabilidades condicionales con el contexto "I like":

$\hat{W_2}=\arg\max_{W_2} \left( P(W_2 | W_{1}=like,W_{0}=I)\right)$

y para las palabras "dogs" y "cats":

$ P(\text{"dogs"} |\text{ "like", "I"}) = \frac{Count(\text{I, like, dogs})}{\text{Total de  ocurrencias de "I", "like"}} = \frac{1}{1} = 1 $

$ P(\text{"cats"} | \text{"like", "I"}) = \frac{Count(\text{I, like, cats})}{\text{Total \ de \ ocurrencias \ de \ "I", "like"}} = 0$

Estas probabilidades indican la probabilidad de encontrar la tercera palabra en un trigrama. Es notable que el resultado $\hat{W_2}$ es "dogs", lo cual parece concordar mejor con la secuencia.

La función ```trigrams``` es proporcionada por la biblioteca NLTK de Python. Esta función toma una secuencia de tokens como entrada, devuelve un iterador sobre tripletes consecutivos de tokens (trigramas) y los convierte en una distribución de frecuencias.


In [ ]:
freq_trigrams = nltk.FreqDist(nltk.trigrams(tokens))
freq_trigrams

Calculamos la probabilidad para cada una de las siguientes palabras.


In [ ]:
make_predictions("so do", freq_trigrams, normlize=freq_bigrams[('do','i')])[0:10]

Calculamos la probabilidad para cada una de las siguientes palabras.


In [ ]:
mi_song = ""

w1 = tokens[0]
for w2 in tokens[0:100]:
    gram = w1 + ' ' + w2
    mi_word = make_predictions(gram, freq_trigrams)[0][0]
    mi_song += " " + mi_word
    w1 = w2

In [ ]:
mi_song

Existen diversos desafíos asociados con los métodos basados en histogramas. Por ejemplo, si consideramos que existen **N** palabras en el vocabulario, un modelo unigrama tendría $N$ compartimientos, mientras que un modelo bigrama tendría $N^2$ compartimientos y así sucesivamente.

Los modelos n-grama también tienen limitaciones en su capacidad para captar el contexto y las relaciones intrincadas entre palabras. 

Por ejemplo, consideremos las frases `I hate dogs`, `I don’t like dogs` y el hecho de que **don’t like** significa **dislike**. En este contexto, un enfoque basado en histogramas fallaría en entender la importancia semántica de la frase **don’t like** que equivale a **dislike**, perdiendo así la relación semántica esencial que implica.


### **Redes neuronales feedforward (FNN) para modelos de lenguaje**

FNNs, o perceptrones multicapa, constituyen los componentes básicos para comprender las redes neuronales en NLP. En tareas de NLP, las FNNs procesan datos textuales transformándolos en vectores numéricos llamados **embeddings**. 

Estos embeddings se introducen en la red para predecir aspectos del lenguaje, como la siguiente palabra de una oración o el sentimiento de un texto.


In [ ]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

#### Tokenización para FNN


Esta función de PyTorch se usa para obtener un tokenizador de texto.


In [ ]:
tokenizer = get_tokenizer("basic_english")
tokens = tokenizer(song)

#### Indexación

TorchText proporciona herramientas para tokenizar el texto en palabras individuales (tokens) y construir un vocabulario, el cual asigna a cada token un índice entero único. Esto es crucial para preparar los datos textuales para modelos de aprendizaje automático que requieren entrada numérica.


In [ ]:
# Paso 1: Creamos un vocabulario a partir de los tokens del texto

# Tokenizamos el texto 'song' usando el tokenizador proporcionado.
# La función map aplica el tokenizador a cada palabra del texto al separarlo.
# El resultado es una lista de tokens que representan las palabras del 'song'.
tokenized_song = map(tokenizer, song.split())

# Paso 2:  Se realiza la construcción del vocabulario
# La función build_vocab_from_iterator construye un vocabulario a partir del texto tokenizado.
# En este caso, se añade un token especial "<unk>" (token desconocido) para manejar palabras fuera del vocabulario.
vocab = build_vocab_from_iterator(tokenized_song, specials=["<unk>"])

# Paso 3: Se establece el índice por defecto
# Se asigna el índice por defecto del vocabulario al índice correspondiente al token "<unk>".
# Esto asegura que cualquier token desconocido en el futuro se mapee a este índice.
vocab.set_default_index(vocab["<unk>"])

Se convierte los tokens a índices aplicando la función, como se muestra a continuación:


In [ ]:
vocab(tokens[0:10])

Se escribe una función de texto que convierta el texto a índices.


In [ ]:
text_pipeline = lambda x: vocab(tokenizer(x))
text_pipeline(song)[0:10]

Se busca la palabra correspondiente a un índice utilizando el método ```get_itos()```. El resultado es una lista en la que el índice de la lista corresponde a una palabra.


In [ ]:
index_to_token = vocab.get_itos()
index_to_token[0]

### **Capa de embeddings**

Una capa de embeddings es un componente crucial en el procesamiento del lenguaje natural (NLP) y en las redes neuronales diseñadas para datos secuenciales. Su función es convertir variables categóricas, como palabras o índices discretos que representan tokens, en vectores continuos. Esta transformación facilita el entrenamiento y permite que la red aprenda relaciones significativas entre palabras.

Consideramos un ejemplo simple con un vocabulario de palabras
- **Vocabulario**: {apple, banana, orange, pear}

Cada palabra en el vocabulario tiene asignado un índice único:
- **Índices**: {0, 1, 2, 3}

Cuando se usa una capa de embeddings, se inicializan vectores continuos de forma aleatoria para cada índice. Por ejemplo, los vectores de embeddings podrían ser:

- Vector para el índice 0 (apple): `[0.2, 0.8]`
- Vector para el índice 1 (banana): `[0.6, -0.5]`
- Vector para el índice 2 (orange): `[-0.3, 0.7]`
- Vector para el índice 3 (pear): `[0.1, 0.4]`

En PyTorch, podemos crear una capa de embeddings de la siguiente manera:


In [ ]:
embedding_dim = 20
vocab_size = len(vocab)
embeddings = nn.Embedding(vocab_size, embedding_dim)

**Embeddings**: Obtenemos el embedding para la primera palabra con índice 0 o 1. 

Es necesario convertir la entrada a un tensor. Los embeddings se inicializan aleatoriamente, pero a medida que el modelo se entrena, las palabras con significados similares tenderán a agruparse.


In [ ]:
for n in range(2): 
    embedding = embeddings(torch.tensor(n))
    print("Palabra", index_to_token[n])
    print("Índice", n)
    print("Embedding", embedding)
    print("Forma del embedding", embedding.shape)

Estos vectores servirán como entrada para la siguiente capa.


#### Generación de pares contexto-objetivo (n-gramas)

Se organiza las palabras dentro de un contexto de tamaño variable utilizando el siguiente enfoque: 

Cada palabra se denota por 'i'. Para establecer el contexto, resta 'j'. El tamaño del contexto viene determinado por el valor de ``CONTEXT_SIZE``.


In [ ]:
CONTEXT_SIZE = 2

ngrams = [
    (
        [tokens[i - j - 1] for j in range(CONTEXT_SIZE)],
        tokens[i]
    )
    for i in range(CONTEXT_SIZE, len(tokens))
]

Se muestra el primer elemento, que resulta en una tupla. El primer elemento representa el contexto y el segundo la palabra objetivo.


In [ ]:
context, target = ngrams[0]
print("Contexto", context, "Objetivo", target)
print("Indice del contexto", vocab(context), "Indice del objetivo", vocab([target]))

En este contexto, hay varias palabras.  Se agrega los embeddings de cada una de estas palabras y se ajusta el tamaño de entrada para la siguiente capa. Luego, se crea la siguiente capa.


In [ ]:
linear = nn.Linear(embedding_dim * CONTEXT_SIZE, 128)

Tenemos dos embeddings.


In [ ]:
mi_embeddings = embeddings(torch.tensor(vocab(context)))
mi_embeddings.shape

Cambiamos la forma de los embeddings.


In [ ]:
mi_embeddings = mi_embeddings.reshape(1, -1)
mi_embeddings.shape

Ahora podemos usar esto como entrada en la siguiente capa.


In [ ]:
linear(mi_embeddings)

#### Función batch

Creamos una función batch para interactuar con el DataLoader. Se requieren varios ajustes para manejar palabras que forman parte de un contexto en un batch y que son la palabra objetivo en el siguiente batch.


In [ ]:
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
CONTEXT_SIZE = 3
BATCH_SIZE = 10
EMBEDDING_DIM = 10

def collate_batch(batch):
    batch_size = len(batch)
    context, target = [], []
    for i in range(CONTEXT_SIZE, batch_size):
        target.append(vocab([batch[i]]))
        context.append(vocab([batch[i - j - 1] for j in range(CONTEXT_SIZE)]))

    return torch.tensor(context).to(device), torch.tensor(target).to(device).reshape(-1)

Del mismo modo, es importante resaltar que el tamaño del último batch podría ser diferente al de los anteriores. Para solucionar esto, se ajusta el último batch para que cumpla con el tamaño especificado, asegurando que sea un múltiplo del tamaño predeterminado. Cuando sea necesario, se utilizarán técnicas de padding (relleno) para lograr esta homogeneidad. Una de las estrategias utilizadas es añadir el comienzo de la canción al final del batch.


In [ ]:
Padding = BATCH_SIZE - len(tokens) % BATCH_SIZE
tokens_pad = tokens + tokens[0:Padding]

Creamos el `DataLoader`.


In [ ]:
dataloader = DataLoader(
     tokens_pad, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_batch
)

### **Redes neuronales multiclase**

Hemos desarrollado una clase en PyTorch para una red neuronal multiclase. La salida de la red es la probabilidad de la siguiente palabra dentro de un contexto dado. Por ello, el número de clases corresponde al número de palabras distintas. La capa inicial consiste en embeddings y, además de la capa final, se incorpora una capa oculta adicional.


In [ ]:
class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.context_size = context_size
        self.embedding_dim = embedding_dim
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs)
        embeds = torch.reshape(embeds, (-1, self.context_size * self.embedding_dim))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)

        return out

Creamos un modelo.


In [ ]:
modelo = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE).to(device)

Extraemos un lote de datos `(context, target)` del DataLoader y pasamos el contexto como entrada al modelo para obtener la salida.

In [ ]:
context, target = next(iter(dataloader))
out = modelo(context)

Aunque el modelo aún no ha sido entrenado, analizar la salida nos permite entender mejor cómo funciona. En la salida, la primera dimensión corresponde al tamaño del batch, mientras que la segunda dimensión representa la probabilidad asociada a cada clase.


In [ ]:
out.shape

Encontramos el índice con la mayor probabilidad.


In [ ]:
predicted_index = torch.argmax(out, 1)
predicted_index

Buscamos el token correspondiente.


In [ ]:
[index_to_token[i.item()] for i in predicted_index]

Creamos una función que realice la misma tarea para los tokens.


In [ ]:
def write_song(modelo, number_of_words=100):
    mi_song = ""
    for i in range(number_of_words):
        with torch.no_grad():
            context = torch.tensor(vocab([tokens[i - j - 1] for j in range(CONTEXT_SIZE)])).to(device)
            word_inx = torch.argmax(modelo(context))
            mi_song += " " + index_to_token[word_inx.detach().item()]

    return mi_song

In [ ]:
write_song(modelo)

#### Entrenamiento

Entrenar un modelo de lenguaje involucra un proceso de múltiples pasos en el que se utilizan datos de entrenamiento y prueba para optimizar el rendimiento del modelo. En el ámbito del procesamiento de lenguaje natural (NLP), este proceso a menudo utiliza diversas métricas para evaluar la precisión del modelo, como la perplexidad o la exactitud en datos no vistos. Sin embargo, en el contexto de esta exploración, emprenderemos un camino ligeramente diferente. En lugar de depender únicamente de las métricas convencionales de NLP, el enfoque se desplaza a la inspección manual de los resultados.

Contamos con la pérdida de entropía cruzada (`cross entropy loss`) entre los logits de entrada y el objetivo:


In [ ]:
criterion = torch.nn.CrossEntropyLoss()

Hemos desarrollado una función dedicada a entrenar el modelo usando el DataLoader proporcionado. Además de entrenar el modelo, la función muestra predicciones para cada época, generando contexto para las siguientes 100 palabras.


In [ ]:
def train(dataloader, modelo, number_of_epochs=100, show=10):
    """
    Args:
        dataloader (DataLoader): DataLoader que contiene los datos de entrenamiento.
        modelo (nn.Module): Modelo de red neuronal a entrenar.
        number_of_epochs (int, opcional): Número de épocas para el entrenamiento. Por defecto es 100.
        show (int, opcional): Intervalo para mostrar el progreso. Por defecto es 10.

    Returns:
        list: Lista que contiene los valores de pérdida para cada época.
    """

    MI_LOSS = []  # Lista para almacenar los valores de pérdida para cada época

    # Itera sobre el número de épocas especificado
    for epoch in tqdm(range(number_of_epochs)):
        total_loss = 0  # Inicializa la pérdida total para la época actual
        mi_song = ""    # Inicializa una cadena para almacenar la canción generada

        # Itera sobre cada batch en el dataloader
        for context, target in dataloader:
            modelo.zero_grad()          # Pone a cero los gradientes para evitar acumulación
            predicted = modelo(context)  # Paso forward a través del modelo para obtener predicciones
            loss = criterion(predicted, target.reshape(-1))  # Calcula la pérdida
            total_loss += loss.item()   # Acumula la pérdida

            loss.backward()    # Backpropagation para calcular los gradientes
            optimizer.step()   # Actualiza los parámetros del modelo usando el optimizador

        # Muestra el progreso y generar una canción cada cierto intervalo
        if epoch % show == 0:
            mi_song += write_song(modelo)  # Genera la canción usando el modelo

            print("Canción generada:")
            print("\n")
            print(mi_song)

        MI_LOSS.append(total_loss / len(dataloader))  # Agrega la pérdida total promedio de la época a la lista MY_LOSS

    return MI_LOSS  # Devuelve la lista de valores promedio de pérdida por época

La siguiente lista se utilizará para almacenar la pérdida de cada modelo.


In [ ]:
mi_loss_list = []

Este segmento de código inicializa un modelo de lenguaje n-grama con un tamaño de contexto de 2. El modelo, denominado `model_2`, se configura en base al tamaño del vocabulario, la dimensión del embedding y el tamaño del contexto proporcionados. 

Se utiliza el optimizador stochastic gradient descent (SGD) con una tasa de aprendizaje de 0.01 para gestionar la actualización de parámetros del modelo. Además, se configura un programador de tasa de aprendizaje (learning rate scheduler) usando un método escalonado con un factor de reducción de 0.1 por época, el cual se encarga de ajustar la tasa de aprendizaje durante el proceso de entrenamiento. 

Estas configuraciones establecen el marco para entrenar el modelo n-grama con optimizaciones y ajustes en la tasa de aprendizaje.


In [ ]:
# Define el tamaño del contexto para el modelo n-grama
CONTEXT_SIZE = 2

# Crea una instancia de la clase NGramLanguageModeler con los parámetros especificados
model_2 = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE).to(device)

# Define el optimizador para entrenar el modelo, usando descenso de gradiente estocástico (SGD)
optimizer = optim.SGD(model_2.parameters(), lr=0.01)

# Configura un programador de tasa de aprendizaje usando StepLR para ajustar la tasa de aprendizaje durante el entrenamiento
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1.0, gamma=0.1)

Ahora, entrenamos el modelo.


In [ ]:
mi_loss = train(dataloader, model_2)

Se guarda el modelo.


In [ ]:
save_path = '2gram.pth'
torch.save(model_2.state_dict(), save_path)
mi_loss_list.append(mi_loss)

El siguiente código muestra los embeddings de palabras del modelo creado, reduce su dimensionalidad a 2D usando t-SNE y luego los grafica en un diagrama de dispersión. Además, se anotan los primeros 20 puntos en la visualización con las palabras correspondientes. Esto se utiliza para visualizar cómo las palabras similares se agrupan en un espacio de menor dimensión, revelando la estructura de los embeddings. 

Los embeddings permiten al modelo representar las palabras en un espacio vectorial continuo, capturando relaciones y similitudes semánticas entre ellas.


In [ ]:
X = model_2.embeddings.weight.cpu().detach().numpy()
tsne = TSNE(n_components=2, random_state=42)
X_2d = tsne.fit_transform(X)

labels = []

for j in range(len(X_2d)):
    if j < 20:
        plt.scatter(X_2d[j, 0], X_2d[j, 1], label=index_to_token[j])
        labels.append(index_to_token[j])
        # Añade la palabra como anotación
        plt.annotate(index_to_token[j],
                     (X_2d[j, 0], X_2d[j, 1]),
                     textcoords="offset points",
                     xytext=(0, 10),
                     ha='center')
    else:
        plt.scatter(X_2d[j, 0], X_2d[j, 1])

plt.legend(labels, loc='upper left', bbox_to_anchor=(1, 1))
plt.show()

Repetimos el proceso para un contexto de cuatro.


In [ ]:
CONTEXT_SIZE = 4
model_4 = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE).to(device)
optimizer = optim.SGD(model_4.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
mi_loss = train(dataloader, model_4 )

save_path = '4gram.pth'
torch.save(model_4.state_dict(), save_path)

mi_loss_list.append(mi_loss)

El código que se muestra a continuación presenta los embeddings de palabras del modelo creado, reduce su dimensionalidad a 2D usando t-SNE y luego los grafica en un diagrama de dispersión. Además, se anotan los primeros 20 puntos en la visualización con las palabras correspondientes. Esto se usa para visualizar cómo las palabras similares se agrupan en un espacio de menor dimensión, revelando la estructura de los embeddings. 

Los embeddings permiten que el modelo represente las palabras en un espacio vectorial continuo, capturando relaciones y similitudes semánticas entre ellas.


In [ ]:
X = model_4.embeddings.weight.cpu().detach().numpy()
tsne = TSNE(n_components=2, random_state=42)
X_2d = tsne.fit_transform(X)

labels = []

for j in range(len(X_2d)):
    if j < 20:
        plt.scatter(X_2d[j, 0], X_2d[j, 1], label=index_to_token[j])
        labels.append(index_to_token[j])
        # Añade la palabra como anotación
        plt.annotate(index_to_token[j],
                     (X_2d[j, 0], X_2d[j, 1]),
                     textcoords="offset points",
                     xytext=(0, 10),
                     ha='center')
    else:
        plt.scatter(X_2d[j, 0], X_2d[j, 1])

plt.legend(labels, loc='upper left', bbox_to_anchor=(1, 1))
plt.show()

Finalmente, para un contexto de ocho.


In [ ]:
CONTEXT_SIZE = 8
model_8 = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE).to(device)
optimizer = optim.SGD(model_8.parameters(), lr=0.01)

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
mi_loss = train(dataloader, model_8)

save_path = '8gram.pth'
torch.save(model_8.state_dict(), save_path)

mi_loss_list.append(mi_loss)

El siguiente código muestra los embeddings de palabras del modelo creado, reduce su dimensionalidad a 2D usando t-SNE y luego los grafica en un diagrama de dispersión. Además, se anotan los primeros 20 puntos de la visualización con las palabras correspondientes. Esto se usa para visualizar cómo se agrupan las palabras similares en un espacio de menor dimensión, revelando la estructura de los embeddings. 

Los embeddings permiten que el modelo represente las palabras en un espacio vectorial continuo, capturando relaciones y similitudes semánticas entre ellas.


In [ ]:
X = model_8.embeddings.weight.cpu().detach().numpy()
tsne = TSNE(n_components=2, random_state=42)
X_2d = tsne.fit_transform(X)

labels = []

for j in range(len(X_2d)):
    if j < 20:
        plt.scatter(X_2d[j, 0], X_2d[j, 1], label=index_to_token[j])
        labels.append(index_to_token[j])
        # Añade la palabra como anotación
        plt.annotate(index_to_token[j],
                     (X_2d[j, 0], X_2d[j, 1]),
                     textcoords="offset points",
                     xytext=(0, 10),
                     ha='center')
    else:
        plt.scatter(X_2d[j, 0], X_2d[j, 1])

plt.legend(labels, loc='upper left', bbox_to_anchor=(1, 1))
plt.show()

Al observar la pérdida graficada para cada modelo, se evidencia que a mayor tamaño del contexto, menor es la pérdida. Aunque este enfoque no incluye validación del modelo ni utiliza métricas convencionales de NLP, la evidencia visual respalda un mejor rendimiento.


In [ ]:
for (mi_loss, model_name) in zip(mi_loss_list, ["2-grama", "4-grama", "8-grama"]):
    plt.plot(mi_loss, label="Pérdida de entropía cruzada - {}".format(model_name))
    plt.legend()

### **Perplexidad**

La perplexidad es una medida utilizada para evaluar la efectividad de modelos de lenguaje o modelos probabilísticos. Proporciona una indicación de qué tan bien un modelo predice una muestra de datos o la probabilidad de un evento no visto. La perplexidad es comúnmente usada en tareas de procesamiento de lenguaje natural, como la traducción automática, el reconocimiento de voz y la generación de lenguaje.

La perplexidad se deriva del concepto de pérdida de entropía cruzada, que mide la disimilitud entre las probabilidades predichas y las reales.

$$\text{Pérdida de entropía cruzada} = -\sum_{i=1}^{N} y_i \ln(p_i)$$

La pérdida de entropía cruzada se calcula tomando la suma negativa del producto de las etiquetas verdaderas $y_i$ y el logaritmo de las probabilidades predichas $p_i$ sobre $N$ clases.

El cálculo del exponencial del promedio de la pérdida de entropía cruzada nos da el valor de la perplexidad.

$$\text{Perplexidad} = e^{\frac{1}{N} \text{Pérdida de entropía cruzada}}$$

Un valor menor de perplexidad indica que el modelo es más confiado y preciso al predecir los datos. Por el contrario, una perplexidad alta sugiere que el modelo es menos certero en sus predicciones.

La perplexidad puede verse como una estimación del número promedio de opciones que el modelo tiene para la siguiente palabra o evento en una secuencia. Una perplexidad menor significa que el modelo está más seguro acerca de la siguiente palabra, mientras que una perplexidad mayor implica que existen más opciones posibles.


In [ ]:
for (mi_loss, model_name) in zip(mi_loss_list, ["2-grama", "4-grama", "8-grama"]):
    # Calcula la perplexidad usando la pérdida
    perplexity = np.exp(mi_loss)
    plt.plot(perplexity, label="Perplexidad - {}".format(model_name))
    plt.legend()

### Ejercicios

1. **Análisis de frecuencias y smoothing**  
   - A partir de un corpus pequeño (p. ej. versos de canciones), calcula manualmente distribuciones unigramas y bigramas.  
   - Aplica al menos dos técnicas de suavizado (Laplace, Good–Turing) y compara cómo cambian las probabilidades de eventos raros.

2. **Evaluación de modelos n‑grama**  
   - Diseña un experimento para medir la *perplexidad* de modelos de orden 1, 2 y 3 sobre un conjunto de validación.  
   - Interpreta qué tamaño de n‑grama equilibra mejor capacidad predictiva y complejidad.

3. **Selección de contexto óptimo**  
   - Plantea una estrategia para elegir el tamaño de ventana (context size) en redes feedforward de lenguaje.  
   - ¿Cómo influye la longitud del contexto en la calidad de las predicciones y la eficiencia del entrenamiento?

4. **Comparación de arquitecturas**  
   - Propón una prueba comparativa entre un modelo n‑grama puro y una red neuronal simple (FNN) en la tarea de completar frases.  
   - Define métricas cuantitativas (precisión, cobertura de vocabulario) y cualitativas (fluidez, coherencia).

5. **Visualización de embeddings**  
   - Elabora un protocolo para proyectar vectores de palabras a 2D (t‑SNE o PCA).  
   - Describe cómo interpretar agrupamientos semánticos y detectar outliers en el espacio de embedding.

6. **Detección de colisiones semánticas**  
   - Identifica pares de palabras muy frecuentes que generen ambigüedad ("bank", "lead2, etc.) y diseña un análisis de contexto para desambiguarlas.  
   - Explica cómo un modelo de mayor orden mejoraría la desambiguación.

7. **Generación controlada de texto**  
   - Plantea un método para "sembrar" (seed) la generación de texto con una palabra o frase inicial y evaluar la diversidad de salidas.  
   - ¿Qué ocurre si restringes el vocabulario del modelo durante la generación?

8. **Impacto del preprocesamiento**  
   - Compara distintas estrategias de limpieza (eliminación de stop‑words, lematización, minúsculas vs. mayúsculas) y analiza su efecto sobre la cobertura del vocabulario y la entropía del modelo.

9. **Análisis de errores**  
   - Reúne ejemplos de predicciones incorrectas de tu modelo n‑grama y clasifícalas según la razón del fallo (datos escasos, ambigüedad, etc.).  
   - Sugiere mejoras de modelado o ampliación de corpus para cada caso.

10. **Proyecto integrador**  
    - Diseña un pequeño pipeline que vaya desde el preprocesamiento hasta la evaluación final de un modelo de lenguaje (incluyendo generación, métricas y visualizaciones).  
    - Detalla los componentes, flujos de datos y criterios de éxito para una entrega académica.

In [ ]:
## Tus respuestas